In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from sklearn import svm
import numpy as np
import shap
import lightgbm as lgb

In [ ]:
import sys
sys.path.append('..')
from util  import Transform, PredictWay, get_k_fold_data
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc 
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
import matplotlib
matplotlib.matplotlib_fname()

In [ ]:
shap.initjs()

In [ ]:
MATRIX = []
def str2Nan(x):
    try:
        return float(x)
    except:
        return np.nan
seed=0
patient = pd.read_excel('../../8.7.xls')
transform = Transform(seed_num=seed)
patient = transform.confuse(patient) 
# X, Y = transform.drop(patient,startXIndex=0,YIndex=2,dropColunm=['肿瘤1，结核4，慢性炎症2，急性炎症3，漏出液0'])    
# X, Y = transform.drop(patient,startXIndex=0,YIndex=3,dropColunm=['恶性1，良性0'])    
X, Y = transform.drop(patient,startXIndex=0,YIndex=-1,dropColunm=['Y'])    
for name in X.columns:
    X[name] = X[name].apply(str2Nan)
    X[name] = X[name].fillna(0)

In [ ]:
k = 5
X_pre = X[:-200].copy()
# X = transform.normalization(X)
result_svm, result_gbdt,result_tree,result_randomtree,result_lg,result_knn,result_lgb = [],[], [], [],[],[],[]
false_index = []
test_x, test_y = X[-200:],Y[-200:]
X, Y = X[:-200],Y[:-200]

In [ ]:
ModelDict = {
#     'SVM':svm.SVC(random_state=seed, probability=True),
#     'DT':DecisionTreeClassifier(max_depth=10,
#                                           random_state=seed),
#     'GBDT':GradientBoostingClassifier(random_state=seed),
#     'RF':RandomForestClassifier(random_state=seed,
#                                           min_samples_leaf=10),
#     'KNN':KNeighborsClassifier(),
#     'LR':LogisticRegression(random_state=400,max_iter=100),
    'LightGBM': lgb.LGBMClassifier(num_leaves=100,learning_rate=0.5,random_state=42,verbosity=-1)}

for i in range(k):
    if i!=4:
        continue
    print('第{:d}代'.format(i+1))
    X_train, y_train, x_valid, y_valid,Y_index = get_k_fold_data(k,i,X,Y,transform.confuse_index)
    pre = PredictWay(X_train, y_train, x_valid, y_valid,seed,Y_index)
    for model_name, model in ModelDict.items():
        matric = pre.model_train(model_name, model)
        if model_name=='LightGBM':
            lgb_model = pre.model_fit
    explainer = shap.TreeExplainer(lgb_model)
    shap_values = explainer.shap_values(pre.trainx)
pre.show_roc()

In [ ]:
shap.summary_plot(shap_values,X_train,show=False)
# plt.savefig('shap_summary_svm.png')

In [ ]:
shap_values.shape

In [ ]:
test_shap_values = explainer.shap_values(test_x)

In [ ]:
X_train.iloc[133]

In [ ]:
shap.summary_plot(shap_values, X_train,show=False)
plt.savefig('shap_summary_scatter0.png')

In [ ]:
shap.plots.force(explainer.expected_value,shap_values)
# plt.savefig('D:/RR.jpg')

In [ ]:
shap_values_bar = explainer(X_train)
shap.plots.bar(shap_values_bar[2], max_display=22)

In [ ]:
shap.summary_plot(shap_values, X_train, 
            plot_type="bar",show=False)
plt.savefig('shap_summary_plt.png')

In [ ]:

plt.figure(figsize=(15, 5))
plt.bar(range(len(X_train.keys())), pre.lgb_model.feature_importances_)
plt.xticks(range(len(X_train.keys())), X_train.keys(), rotation=-45, fontsize=14)
plt.title('Feature importance', fontsize=14)
plt.show()

In [ ]:
shap.dependence_plot(('Age','Age'),explainer.shap_interaction_values(X),X_pre,show=False,xmax = 100)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('Age(year)')
plt.savefig('shap_dependence_Age.png')


shap.dependence_plot(('Eosinophil','Eosinophil'),explainer.shap_interaction_values(X),X_pre,show=False,xmax = 100)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('Eosinophil (%)')
plt.savefig('shap_dependence_Eosinophil.png')

shap.dependence_plot(('TB Ab','TB Ab'),explainer.shap_interaction_values(X),X_pre,show=False,xmax = 2)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('TB Ab')
plt.savefig('shap_dependence_TB Ab.png')

shap.dependence_plot(('ADA','ADA'),explainer.shap_interaction_values(X),X_pre,show=False,xmax = 220)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('ADA (U/L)')
plt.savefig('shap_dependence_ADA.png')

shap.dependence_plot(['Glucose','Glucose'],explainer.shap_interaction_values(X),X_pre,show=False,xmax=150)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('Glucose (mmol/L)')
plt.savefig('shap_dependence_Glucose.png')

shap.dependence_plot(('Chloride','Chloride'),explainer.shap_interaction_values(X),X_pre,show=False,xmax=150)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('Chloride(mmol/L)')
plt.savefig('shap_dependence_Chloride.png')

shap.dependence_plot(('LDH','LDH'),explainer.shap_interaction_values(X),X_pre,show=False,xmax=16000)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('LDH(U/L)')
plt.savefig('shap_dependence_LDH.png')

shap.dependence_plot(('Protein','Protein'),explainer.shap_interaction_values(X),X_pre,show=False,xmax=80)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('Protein(mg/dL)')
plt.savefig('shap_dependence_Protein.png')

shap.dependence_plot(['CA153','CA153'],explainer.shap_interaction_values(X),X_pre,show=False)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('CA153 (U/mL)')
plt.savefig('shap_dependence_CA153.png')

shap.dependence_plot(['CK19','CK19'],explainer.shap_interaction_values(X),X_pre,show=False,xmax=100)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('CK19 (ng/mL)')
plt.savefig('shap_dependence_CK19.png')

shap.dependence_plot(('SCC','SCC'),explainer.shap_interaction_values(X),X_pre,show=False)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')
plt.xlabel('SCC(ng/mL)')
plt.savefig('shap_dependence_SCC.png')





In [ ]:
shap.dependence_plot(
    ("Age", "Age"),
    explainer.shap_interaction_values(X[:500]),
    X_pre[:500], show=False,dot_size=1)
plt.grid(ls=":",c='gray')
plt.axhline(0,ls = '-.', c='r')